In [197]:
import os
import json
import pandas as pd
import numpy as np
import shutil
import random
import pickle
from tqdm import tqdm
from pathlib import Path

import sys
sys.path.insert(1, '../network_analysis/')
import utils

# Functions

In [198]:
def get_val_to_col_name_dict(input_dir):
    '''
    Given an input directory containing the raw csv files return the `val_to_col_name_dict`
    '''
    val_to_col_name_dict = {}
    # Find the all Homographs and populate the `val_to_col_name_dict`
    for filename in tqdm(os.listdir(input_dir)):
        df = pd.read_csv(input_dir+filename, keep_default_na=False, dtype=str)
        column_names=list(df.columns)
        for idx, row in df.iterrows():
            for i in range(len(row.tolist())):
                if row[i] in val_to_col_name_dict:
                    val_to_col_name_dict[row[i]].add(column_names[i])
                else:
                    val_to_col_name_dict[row[i]] = set([column_names[i]])

    return val_to_col_name_dict

def get_homographs_and_clean_dataset(input_dir, output_dir, column_name_to_homograph_type, val_to_col_name_dict):
    '''
    Identifies all homographs and outputs a dataframe with them at `output_dir`.
    Also creates a new version on the input tables without he inclusion of the homographs and outputs them at `output_dir/no_homographs/`

    Returns the `homograph_info_df` for all types homographs
    '''
    Path(output_dir+'no_homographs/').mkdir(parents=True, exist_ok=True)
    homograph_info_dict={'value': [], 'filename': [], 'column_name' : [], 'type': [], 'subtype': [], "contents_row": []}

    for filename in tqdm(os.listdir(input_dir)):
        df = pd.read_csv(input_dir+filename, keep_default_na=False, dtype=str)
        column_names=list(df.columns)

        row_ids_with_homographs=set()
        for idx, row in df.iterrows():
            for i in range(len(row.tolist())):
                # Check if value is a homograph
                if row[i]=='':
                    row_ids_with_homographs.add(idx)
                elif len(val_to_col_name_dict[row[i]])>1:
                    row_ids_with_homographs.add(idx)
                    homograph_info_dict['value'].append(row[i])
                    homograph_info_dict['filename'].append(filename)
                    homograph_info_dict['column_name'].append(column_names[i])
                    homograph_info_dict['type'].append(column_name_to_homograph_type[column_names[i]]['type'])
                    homograph_info_dict['subtype'].append(column_name_to_homograph_type[column_names[i]]['subtype'])
                    homograph_info_dict['contents_row'].append(row.tolist())
            
        df_no_homographs = df.drop(labels=row_ids_with_homographs)
        df_no_homographs.to_csv(output_dir+'no_homographs/'+filename,index=False)

    homograph_info_df = pd.DataFrame.from_dict(homograph_info_dict)
    
    return homograph_info_df

def select_homographs_of_specified_type(homograph_df, num_homographs=10, type='traditional', subtype=None, seed=0):
    '''
    Returns a list of the selected homographs from the specified types. 

    Raises an error if it is not possible to extract the specified types
    '''

    # Check if the requested specification is possible (i.e., there are enough unique homographs)
    valid_tuples_df=homograph_df[homograph_df['type']==type]
    if subtype: valid_tuples_df=homograph_df[homograph_df['subtype']==subtype]
    if valid_tuples_df['value'].nunique() < num_homographs:
        raise ValueError('Not possible to extract ' + str(num_homographs) + ' homographs. There are only ' + str(valid_tuples_df['value'].nunique()) + ' available homographs with the specified parameters.')

    # Select the homographs
    np.random.seed(seed)
    selected_homographs=np.random.choice(valid_tuples_df['value'].unique(), size=num_homographs, replace=False)
    return selected_homographs

def construct_homograph_injected_dataset(input_dir, output_dir, homograph_info_df, num_homographs=10, type='traditional', subtype=None, seed=0, mode=None):
    '''
    Construct injected homograph datasets of the specified type and amount

    `input_dir`: Directory that contains the tables without any homographs
    `output_dir`: Directory where the injected homographs dataset is stored
    `homograph_info_df`: dataframe that contains the homograph information, types, subtypes, files location etc.
    `num_homographs`: number of unique homographs to be inserted overall
    `type`: the type of homographs to be inserted
    `subtype`: the subtype of homographs to be inserted
    `seed`: seed used for the random selection of homographs from the set of valid homographs
    `mode`: if specified, then all types are considered, for each type `num_homographs` are selected

    Returns a list of the selected homographs
    '''
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    if mode=='all_types':
        # Select homographs from each type
        selected_homographs=[]
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='traditional', subtype=None, seed=seed))
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='symbolic', subtype='code', seed=seed))
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='symbolic', subtype='numeric', seed=seed))
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='null_equivalent', subtype=None, seed=seed))
    else:
        selected_homographs=select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type=type, subtype=subtype, seed=seed)

    valid_tuples_df=select_valid_tuples(homograph_info_df=homograph_info_df, selected_homographs=selected_homographs, type=type, subtype=subtype, mode=mode)
    files_to_modify=set(valid_tuples_df['filename'].unique())
    for filename in tqdm(os.listdir(input_dir)):
        if filename in files_to_modify:
            # Perform insertion
            orig_file_df=pd.read_csv(input_dir+filename, keep_default_na=False, dtype=str)
            tuples_arr=[]
            for idx, row in valid_tuples_df[valid_tuples_df['filename']==filename].iterrows():
                tuples_arr.append(row['contents_row'])
            new_file_df = pd.concat([orig_file_df, pd.DataFrame(tuples_arr, columns=orig_file_df.columns)])
            new_file_df.to_csv(output_dir+filename, index=False)
        else:
            # Copy the file
            shutil.copyfile(input_dir+filename, output_dir+filename)

    return list(selected_homographs)

def select_valid_tuples(homograph_info_df, selected_homographs, type, subtype, mode):
    '''
    Returns a subset of the valid tuples from the specified set
    '''
    if mode=='all_types':
        return homograph_info_df[homograph_info_df['value'].isin(selected_homographs)]
    elif type=='traditional':
        return homograph_info_df[(homograph_info_df['value'].isin(selected_homographs)) & (homograph_info_df['type']==type)]
    elif type=='null_equivalent':
        return homograph_info_df[(homograph_info_df['value'].isin(selected_homographs)) & (homograph_info_df['type']==type)]
    elif type=='symbolic':
        return homograph_info_df[(homograph_info_df['value'].isin(selected_homographs)) & (homograph_info_df['subtype']==subtype)]


def generate_null_equivalent_tuples(input_dir, homograph_info_df, num_meanings=10, num_values=10, seed=0):
    '''
    Returns a dataframe with with content tuples to be used for null-equivalent values

    `input_dir`: input directory containing the raw csv files to be used for the generation
    `homograph_info_df`: dataframe that contains the homograph information, types, subtypes, files location etc., used to ensure no null-equivalent value is found in that dataframe
    `num_meanings`: number of meanings of the generated null_equivalent values (i.e., number of unique column_names selected)
    `num_values`: number of unique null-equivalent values to be formed
    
    Returns a dataframe with the tuples for the null equivalent values
    '''
    random.seed(seed)

    # The string values of the null equivalent values
    null_equivalent_vals = ['null_equivalent_val_'+str(i) for i in range(num_values)]

    # Dictionary mapping each column_name to list of filenames containing them
    column_name_to_filenames_dict={}
    for filename in os.listdir(input_dir):
        columns=pd.read_csv(input_dir+filename, keep_default_na=False, nrows=0, dtype=str).columns.to_list()
        for col in columns:
            if col not in column_name_to_filenames_dict:
                column_name_to_filenames_dict[col]=[filename]
            else:
                column_name_to_filenames_dict[col].append(filename)
    
        # Select `num_meanings` (filename, column_name) pairs for each injected_null_equivalent_val
    null_val_to_filename_column_name_pairs={}
    for val in null_equivalent_vals:
        selected_column_names=random.choices(list(column_name_to_filenames_dict.keys()), k=num_meanings)
        filename_column_name_tuples=[(random.choice(column_name_to_filenames_dict[column_name]), column_name) for column_name in selected_column_names]
        null_val_to_filename_column_name_pairs[val]=filename_column_name_tuples
    
    # Construct the tuples and populate the tuples_dict
    tuples_dict={'value': [], 'filename': [], 'column_name' : [], 'type': [], 'subtype': [], "contents_row": []}
    for val in tqdm(null_equivalent_vals):
        for pair in null_val_to_filename_column_name_pairs[val]:
            filename, column_name=pair[0], pair[1]
            df=pd.read_csv(input_dir+filename, keep_default_na=False, dtype=str)

            # Sample a row and modify the value from the selected column_name
            row = df.sample(n=1, random_state=seed)
            row[column_name]=val

            # Update the tuples_dict
            tuples_dict['value'].append(val)
            tuples_dict['filename'].append(filename)
            tuples_dict['column_name'].append(column_name)
            tuples_dict['type'].append('null_equivalent')
            tuples_dict['subtype'].append(None)
            tuples_dict['contents_row'].append(row.values[0])
    
    return pd.DataFrame.from_dict(tuples_dict)

def get_df_subset_from_mode(df, mode='homographs_traditional'):
    if mode=='homographs_traditional':
        return df[df['type']=='traditional']
    elif mode=='homographs_symbolic_code':
        return df[df['subtype']=='code']
    elif mode=='homographs_symbolic_numeric':
        return df[df['subtype']=='numeric']

def populate_homograph_info_dict(homograph_info_dict):
    all_types_df=homograph_info_dict['homographs_all']
    modes=['homographs_traditional', 'homographs_symbolic_code', 'homographs_symbolic_numeric']

    for mode in tqdm(modes):
        cur_df=get_df_subset_from_mode(df=all_types_df, mode=mode)
        selected_vals=[]
        for val in cur_df['value'].unique():
            if cur_df[cur_df['value']==val]['column_name'].nunique()>1:
                selected_vals.append(val)
        homograph_info_dict[mode]=cur_df[cur_df['value'].isin(selected_vals)]      

    return homograph_info_dict

# Homograph Injection and Dataset Construction

In [204]:
raw_input_dir='../DATA/synthetic_benchmark_large3/'
output_dir='datasets/synthetic_benchmark_large3/'

with open('column_name_to_homograph_type.json') as f:
    column_name_to_homograph_type = json.load(f)

# Dictionary mapping each value to a list of column names
val_to_col_name_dict = get_val_to_col_name_dict(input_dir=raw_input_dir)

# Construct the homograph_info_dict and populate with all types of homographs
homograph_info_dict={}
homograph_info_dict['homographs_all']=get_homographs_and_clean_dataset(output_dir=output_dir, input_dir=raw_input_dir, column_name_to_homograph_type=column_name_to_homograph_type, val_to_col_name_dict=val_to_col_name_dict)
homograph_info_dict = populate_homograph_info_dict(homograph_info_dict)

100%|██████████| 3/3 [00:09<00:00,  3.09s/it]


In [205]:
print("The input dataset contains:", homograph_info_dict['homographs_all']['value'].nunique(), "unique all homograph values.")
print("There are:", homograph_info_dict['homographs_traditional']['value'].nunique(), 'unique traditional only homographs', 
homograph_info_dict['homographs_symbolic_code']['value'].nunique(), 'unique symbolic(code) only homographs',
homograph_info_dict['homographs_symbolic_numeric']['value'].nunique(), 'unique symbolic(numeric) only homographs')

The input dataset contains: 3654 unique all homograph values.
There are: 758 unique traditional only homographs 1138 unique symbolic(code) only homographs 1358 unique symbolic(numeric) only homographs


In [206]:
# Append the null_tuples_df into the homograph_info_df
null_tuples_df = generate_null_equivalent_tuples(input_dir=raw_input_dir, homograph_info_df=homograph_info_dict['homographs_all'], num_meanings=10, num_values=100)
homograph_info_dict['homographs_all']=pd.concat([homograph_info_dict['homographs_all'], null_tuples_df])
with open('datasets/synthetic_benchmark_large3/homograph_info_dict.pickle', 'wb') as handle:
    pickle.dump(homograph_info_dict, handle)

100%|██████████| 100/100 [00:02<00:00, 44.48it/s]


## Injected Datasets Construction

In [207]:
# homograph_info_df=pd.read_pickle(output_dir+'homograph_info_df.pickle')
with open('datasets/synthetic_benchmark_large3/homograph_info_dict.pickle', 'rb') as handle:
    homograph_info_dict = pickle.load(handle)
num_homographs=100
modes=['homographs_traditional', 'homographs_symbolic_code', 'homographs_symbolic_numeric']
homs_dict = {}
for mode in modes:
    subtype=None
    if mode=='homographs_traditional':
        type='traditional'
    elif mode=='homographs_symbolic_code':
        type='symbolic';subtype='code'
    elif mode=='homographs_symbolic_numeric':
        type='symbolic';subtype='numeric'
    homs = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+mode+'_'+str(num_homographs)+'/', homograph_info_df=homograph_info_dict[mode], num_homographs=num_homographs, type=type, subtype=subtype, seed=0)
    homs_dict[mode]=homs
    
homs_dict['homographs_null_equivalent'] = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_null_equivalent_'+str(num_homographs)+'/', homograph_info_df=homograph_info_dict['homographs_all'], num_homographs=num_homographs, type='null_equivalent', seed=0)

# TODO: Ensure correct insertion for all types homographs (ensuring exclusive per type insertion)
homs_dict['homographs_all'] = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_all_'+str(num_homographs)+'/', homograph_info_df=homograph_info_dict['homographs_all'], num_homographs=num_homographs, type=None, seed=0, mode='all_types')

with open(output_dir+'selected_homographs.json', 'w') as f:
    json.dump(homs_dict, f, indent=4)

100%|██████████| 204/204 [00:01<00:00, 144.88it/s]


# Dataset Statistics

In [212]:
import pickle
with open('../graph_construction/combined_graphs_output/synthetic_benchmark_large3/homographs_symbolic_numeric_100/bipartite/bipartite.graph', 'rb') as f:
    G=pickle.load(f)
print(G.number_of_nodes(), G.number_of_edges())

118065 181723


In [213]:
utils.get_attribute_of_instance(G, 'Mallard')

['animal_name_row_id_animal_name_scientific_name_2.csv',
 'street_name_street_name_street_number_postal_code_10.csv',
 'street_name_street_name_street_number_postal_code_11.csv',
 'street_name_street_name_street_number_postal_code_4.csv']

# Testing

In [209]:
homograph_info_dict['homographs_all'][homograph_info_dict['homographs_all']['value']=='Mallard']

,value,filename,column_name,type,subtype,contents_row
926,Mallard,row_id_animal_name_scientific_name_1.csv,animal_name,traditional,None,"[939, Mallard, Anas platyrhynchos]"
1846,Mallard,row_id_animal_name_scientific_name_2.csv,animal_name,traditional,None,"[868, Mallard, Anas platyrhynchos]"
4994,Mallard,row_id_animal_name_scientific_name_6.csv,animal_name,traditional,None,"[62, Mallard, Anas platyrhynchos]"
7895,Mallard,row_id_animal_name_scientific_name_9.csv,animal_name,traditional,None,"[2, Mallard, Anas platyrhynchos]"
8387,Mallard,row_id_animal_name_scientific_name_9.csv,animal_name,traditional,None,"[492, Mallard, Anas platyrhynchos]"
8894,Mallard,row_id_animal_name_scientific_name_10.csv,animal_name,traditional,None,"[10, Mallard, Anas platyrhynchos]"
8997,Mallard,row_id_animal_name_scientific_name_10.csv,animal_name,traditional,None,"[112, Mallard, Anas platyrhynchos]"
9836,Mallard,row_id_animal_name_scientific_name_10.csv,animal_name,traditional,None,"[964, Mallard, Anas platyrhynchos]"
9906,Mallard,animal_name_scientific_name_country_1.csv,animal_name,traditional,None,"[Mallard, Anas platyrhynchos, Indonesia]"
9924,Mallard,animal_name_scientific_name_country_2.csv,animal_name,traditional,None,"[Mallard, Anas platyrhynchos, China]"


In [94]:
with open('datasets/synthetic_benchmark_large3/selected_homographs.json') as f:
    selected_homographs = json.load(f)
selected_homographs['homographs_symbolic_code']

['300',
 '600',
 '2156',
 'Ohio',
 '1444',
 '2712',
 '1185',
 '2434',
 'S8',
 '1605',
 'NC',
 '960',
 'IL',
 '1234',
 '4900',
 'Ghost',
 '525',
 'Florida',
 '4655',
 'MD',
 '2513',
 '2625',
 '1360',
 '6000',
 '1410',
 'MINI',
 '5600',
 'Lightning',
 '5509',
 '3450',
 '4313',
 '1160',
 '6017',
 '1171',
 'LS',
 'ID',
 '3600',
 '3700',
 '3010',
 'Country',
 'Charade',
 'IS',
 '1110',
 'MN',
 '944',
 'SRX',
 'XK',
 '545',
 '4446',
 'Western',
 '1679',
 'Shadow',
 '1696',
 '1009',
 'SD',
 'Forester',
 'MS',
 'Continental',
 '2087',
 '3543',
 'SSR',
 '2618',
 '4513',
 '2407',
 'CO',
 '2630',
 '3165',
 '1304',
 'Ion',
 '4000',
 'KY',
 'AZ',
 'ME',
 '650',
 '6700',
 'Crossfire',
 '5430',
 '3205',
 '215',
 '228',
 'DE',
 '6201',
 '745',
 '4307',
 '7012',
 '1100',
 '7600',
 'Ram',
 '1260',
 '11600',
 '1325',
 'Fury',
 '2270',
 '1040',
 '1427',
 'STS',
 'Washington',
 'Outlander',
 '5309',
 'Arizona']

In [27]:
with open('datasets/synthetic_benchmark_large3/selected_homographs.json') as f:
    selected_homs = json.load(f)
len(selected_homs['homographs_all'])

400

In [208]:
df = pd.read_pickle('evaluation/synthetic_benchmark_large3/eval_dfs.pickle')['homographs_symbolic_numeric'].iloc[:100]
df.head(60)

,node,node_type,approximate_betweenness_centrality,is_homograph,homograph_mode,precision,recall,f1_score
111937,Mallard,cell,0.122995,False,NaN,0.000000,0.00,0.000000
9142,Summit,cell,0.122180,False,NaN,0.000000,0.00,0.000000
1462,0,cell,0.122105,False,NaN,0.000000,0.00,0.000000
9502,850,cell,0.113136,True,homographs_symbolic_numeric,0.250000,0.01,0.019231
9138,1000,cell,0.058254,False,NaN,0.200000,0.01,0.019048
963,1676,cell,0.042134,True,homographs_symbolic_numeric,0.333333,0.02,0.037736
2601,2000,cell,0.029178,True,homographs_symbolic_numeric,0.428571,0.03,0.056075
443,6,cell,0.016583,True,homographs_symbolic_numeric,0.500000,0.04,0.074074
454,5,cell,0.016583,True,homographs_symbolic_numeric,0.555556,0.05,0.091743
9422,98,cell,0.015934,False,NaN,0.500000,0.05,0.090909


In [56]:
np.random.seed(0)
np.random.choice(homograph_info_df[(homograph_info_df['type']=='traditional')]['value'].unique(), size=10, replace=False)

array(['Colorado', 'Duff', 'Elan', 'Garvey', 'Elmira', 'Berkeley', 'Ram',
       'Charity', 'California', 'Crossfire'], dtype=object)

In [30]:
homograph_info_df[homograph_info_df['type']=='symbolic']['value'].unique()

array(['ID', 'NE', 'GT', 'AR', 'CO', 'MA', 'CA', 'DE', 'TL', 'MN', 'AL',
       'SD', 'PA', 'AZ', 'TN', 'CT', 'SC', 'IL', 'GA', 'MD', 'ME',
       'Colorado', 'Elan', 'ES', 'Crossfire', 'Ram', 'California',
       'Jimmy', nan], dtype=object)

In [37]:
val_to_col_name_dict[7]

{'airport_elevation', 'cost', 'row_id'}

In [50]:
homs_tmp=get_homographs_and_clean_dataset(output_dir='../DATA/tmp/', input_dir='../DATA/testing/', column_name_to_homograph_type=column_name_to_homograph_type, val_to_col_name_dict=val_to_col_name_dict)
homs_tmp

100%|██████████| 12/12 [00:00<00:00, 12.49it/s]


,value,filename,column_name,type,subtype,contents_row
0,0,row_id_airport_code_continent_code_airport_ele...,airport_code,symbolic,code,"[0, Usina Mandu Airport, 1597]"
1,10,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[CCK, Cocos (Keeling) Islands Airport, 10]"
2,0,row_id_airport_code_continent_code_airport_ele...,airport_code,symbolic,code,"[0, Venâncio Aires Airport, 226]"
3,100,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[NDA, Bandanaira Airport, 100]"
4,10,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[ARD, Mali Airport, 10]"
...,...,...,...,...,...,...
1139,900,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[ZZV, Zanesville Municipal Airport, 900]"
1140,13,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[ARO, Arboletes Airport, 13]"
1141,626,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[LNN, Willoughby Lost Nation Municipal Airport..."
1142,9,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[FID, Elizabeth Field, 9]"


In [55]:
homs_tmp['value'].unique()

array(['0', '10', '100', '9', '13', '200', '90', '15', '650', '98', '164',
       '545', '42', '760', 'RVR', '325', '740', 'DNA', '88', '940', 'SIS',
       '3500', 'MKZ', '80', '300', '1500', 'TSX', '207', '1', '6000',
       '928', 'MPV', '120', '228', 'LHS', '57', '600', '745', 'SRX',
       '626', 'LUV', 'GTO', '430', '323', 'SLX', '2500', '750', '924',
       'GLI', '550', '62', '911', 'SSR', 'LOL', '960', '900', '9000',
       '525', '240', '500', '944', '330', 'MKS', '645', 'STS', '1000',
       'GTI', '968', 'APV', '914', '929', 'DBS', 'GLC', 'LSS', 'RSX'],
      dtype=object)